In [ ]:
# # !pip install mysql-connector-python
# !pip uninstall -y mysql-connector mysql-connector-python
# !pip install mysql-connector-python==8.0.33


In [1]:
import pandas
import os
import mysql.connector
import json
from datetime import datetime
from mysql.connector import Error



In [4]:
try:
    conn = mysql.connector.connect(
        host="127.0.0.1",
        port=3306,
        user="root",
        password="your_password",
        database="ipl_data",
        ssl_disabled=True
    )

    cursor = conn.cursor()
    print("✅ MySQL connection established successfully.")

except Error as e:
    print("❌ Error while connecting to MySQL:")
    print(f"Error Code: {e.errno}")
    print(f"SQLSTATE: {e.sqlstate}")
    print(f"Message: {e.msg}")

✅ MySQL connection established successfully.


In [6]:

def make_team_id(team_name):
    if not team_name:
        return "UNKNOWN-Team"
    return '.'.join([x[0] for x in team_name.split(" ") if x]) + "-Team"


DATA_DIR = r"D:\your_path\IPL_10"

try:
    for filename in os.listdir(DATA_DIR):
        if filename.endswith(".json"):
            try:
                filepath = os.path.join(DATA_DIR, filename)
                with open(filepath, "r") as file:
                    data = json.load(file)

                match_id = filename.replace(".json", "")

                info = data.get("info", {})
                registry_people = info.get("registry", {}).get("people", {})
                teams_list = info.get("teams", [])
                players_dict = info.get("players", {})
                officials_dict = info.get("officials", {})

                # ========== TEAMS ==========
                current_section = "TEAMS"
                teams_rows = []
                for ind_team in teams_list:
                    team_id = make_team_id(ind_team)
                    teams_rows.append((team_id, ind_team))
                    
                if teams_rows:
                    cursor.executemany("""
                        INSERT IGNORE INTO teams (team_id, team_name)
                        VALUES (%s, %s)
                    """, teams_rows)
                
                print("Inserted TEAMS TABLE")

                # ========== PLAYERS & PLAYERS_TEAM ==========
                current_section = "PLAYERS & PLAYERS_TEAM"
                players_rows = []
                players_team_rows = []
                
                for ind_team in teams_list:
                    for ind_player in players_dict.get(ind_team, []):
                        player_id = registry_people.get(ind_player)
                        if player_id is None:
                            print(f"Warning: player_id missing for {ind_player} in file {filename}")
                            continue
                        players_rows.append((player_id, ind_player))
                        ind_team_id = make_team_id(ind_team)
                        players_team_rows.append((player_id, ind_team_id, info.get('season')))
                        
                if players_rows:
                    cursor.executemany("""
                        INSERT IGNORE INTO players (player_id, player_name)
                        VALUES (%s, %s)
                    """, players_rows)
                    
                if players_team_rows:
                    cursor.executemany("""
                        INSERT IGNORE INTO players_team (player_id, team_id, season)
                        VALUES (%s, %s, %s)
                    """, players_team_rows)
                    
                print("Inserted PLAYERS & PLAYERS_TEAM TABLES")

                # ========== MATCH DETAILS ==========

                current_section = "MATCH DETAILS"
                event = info.get("event", {})
                toss = info.get("toss", {})
                outcome = info.get("outcome", {})

                match_date_str = (info.get('dates') or [None])[0]
                match_date = datetime.strptime(match_date_str, "%Y-%m-%d") if match_date_str else None

                match_number = int(event.get('match_number', 0))
                balls_per_over = int(info.get('balls_per_over', 6))
                overs = int(info.get('overs', 20))
                teams = teams_list or [None, None]
                team_1, team_2 = (teams[0] or "").strip(), (teams[1] or "").strip()
                team1_id = make_team_id(team_1)
                team2_id = make_team_id(team_2)
                winner_team = (outcome.get('winner') or "").strip() or None
                winner_team_id = make_team_id(winner_team) if winner_team else None

                player_of_match = (info.get('player_of_match') or [None])[0]
                player_of_match_id = registry_people.get(player_of_match)
                toss_winner_team_id = make_team_id(toss.get('winner'))

                cursor.execute("""
                    INSERT INTO match_detail (
                        match_id, match_date, city, venue, match_number, stage, match_type, gender, 
                        event_name, balls_per_over, overs, season, team_type,
                        team1_id, team2_id, toss_winner_team_id, toss_decision, winner_team_id,
                        win_by_runs, win_by_wickets, player_of_match_id
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    match_id, match_date, info.get('city'), info.get('venue'), match_number,
                    event.get('stage'), info.get('match_type'), info.get('gender', '').lower(),
                    event.get('name'), balls_per_over, overs, info.get('season'), info.get('team_type'),
                    team1_id, team2_id, toss_winner_team_id, toss.get('decision'), winner_team_id,
                    outcome.get('by', {}).get('runs'), outcome.get('by', {}).get('wickets'),
                    player_of_match_id
                ))
                print("Inserted MATCH DETAILS TABLE")

                # ========== MATCH PLAYERS ==========
                current_section = "MATCH PLAYERS"
                match_players_rows = []
                for ind_team in teams_list:
                    for ind_player in players_dict.get(ind_team, []):
                        player_id = registry_people.get(ind_player)
                        team_id = make_team_id(ind_team)
                        match_players_rows.append((match_id, player_id, team_id))
                if match_players_rows:
                    cursor.executemany("""
                        INSERT INTO match_players (match_id, player_id, team_id)
                        VALUES (%s, %s, %s)
                    """, match_players_rows)

                # ========== OFFICIALS ==========
                current_section = "OFFICIALS"
                officials_rows = []
                for role_list in officials_dict.values():
                    for name in role_list:
                        official_id = registry_people.get(name)
                        officials_rows.append((official_id, name))
                if officials_rows:
                    cursor.executemany("""
                        INSERT IGNORE INTO officials (official_id, official_name)
                        VALUES (%s, %s)
                    """, officials_rows)
                print("Inserted OFFICIALS TABLE")

                # ========== DELIVERIES, POWERPLAY, REVIEWS, REPLACEMENTS, WICKETS ==========
                current_section = "DELIVERIES, POWERPLAY, REVIEWS, REPLACEMENTS, WICKETS"
                deliveries_rows = []
                reviews_rows = []
                replacements_rows = []
                wickets_rows = []
                substitute_players_rows = []
                substitute_players_team = []
                powerplays = []

                innings = data.get('innings', [])
                avv = '_vs_'.join([''.join(word[0] for word in team.split()) for team in teams_list])

                for inning_num, inning in enumerate(innings, start=1):
                    batting_team = inning.get('team')
                    batting_team_id = make_team_id(batting_team)
                    bowling_team = next((t for t in teams_list if t != batting_team), None)
                    bowling_team_id = make_team_id(bowling_team)

                    # POWERPLAY

                    all_powerplay = inning.get('powerplays', [])
                    if all_powerplay:
                        for ind_powerplay in all_powerplay:
                            start_over = int(ind_powerplay.get('from')) + 1
                            end_over = int(ind_powerplay.get('to')) + 1
                            pp_type = ind_powerplay.get('type')
                            if pp_type:
                                powerplay_id = f"{match_id}_{inning_num}_{start_over}_{end_over}_{pp_type[0]}"
                            else:
                                powerplay_id = f"{match_id}_{inning_num}_{start_over}_{end_over}"
                            powerplays.append((powerplay_id, match_id, inning_num, start_over, end_over, pp_type))

                    for over_data in inning.get('overs', []):
                        over_num = int(over_data.get('over'))
                        for ball_num, ind_delivery in enumerate(over_data.get('deliveries', []), start=1):
                            batsman_id = registry_people.get(ind_delivery.get('batter'))
                            bowler_id = registry_people.get(ind_delivery.get('bowler'))
                            non_striker_id = registry_people.get(ind_delivery.get('non_striker'))

                            delivery_id = f"{avv}_{match_id}_{over_num}_{ball_num}_{inning_num}"
                            runs_info = ind_delivery.get('runs', {})
                            runs_batsman = int(runs_info.get('batter', 0))
                            runs_extras = int(runs_info.get('extras', 0))
                            runs_total = int(runs_info.get('total', 0))
                            extras = ind_delivery.get('extras', {})
                            extras_type = next(iter(extras), None)
                            extras_runs = int(extras.get(extras_type, 0)) if extras_type else 0

                            deliveries_rows.append((
                                delivery_id, match_id, inning_num, batting_team_id, bowling_team_id,
                                over_num, ball_num, batsman_id, bowler_id, non_striker_id,
                                runs_batsman, runs_extras, runs_total, extras_type, extras_runs
                            ))

                            # REVIEWS
                            review = ind_delivery.get('review', {})
                            if review.get('by'):
                                review_id = f"{delivery_id}_REVIEW"
                                review_by_team_id = make_team_id(review.get('by'))
                                reviews_rows.append((
                                    review_id, match_id, delivery_id, review_by_team_id,
                                    registry_people.get(review.get('umpire')),
                                    registry_people.get(review.get('batter')),
                                    review.get('decision'), review.get('type'),
                                    review.get('umpires_call')
                                ))

                            # REPLACEMENTS
                            all_replaceent = ind_delivery.get('replacements', {}).get('match', [])
                            if all_replaceent:
                                for i, ind_rep_player in enumerate(all_replaceent, start=1):
                                    replacement_id = f"{delivery_id}_REPL_{i}"
                                    team_id = make_team_id(ind_rep_player.get('team'))
                                    replacements_rows.append((
                                        replacement_id, match_id, team_id,
                                        registry_people.get(ind_rep_player.get('in')),
                                        registry_people.get(ind_rep_player.get('out')),
                                        ind_rep_player.get('reason')
                                    ))

                            # WICKETS
                            all_wickets = ind_delivery.get('wickets', [])
                            if all_wickets:
                                for i, wkt in enumerate(all_wickets, start=1):
                                    wicket_id_base = f"{delivery_id}_WKT_{i}"
                                    player_dismissed_id = registry_people.get(wkt.get("player_out"))
                                    dismissal_kind = wkt.get('kind')
                                    fielders = wkt.get('fielders') or [{}]
                                    for j, ind_fielder in enumerate(fielders, start=1):
                                        fielder_name = ind_fielder.get('name')
                                        fielder_id = registry_people.get(fielder_name) if fielder_name else None
                                        if ind_fielder.get("substitute", False):
                                            substitute_players_rows.append((fielder_id, fielder_name, True))
                                            season = info.get('season')
                                            substitute_players_team.append((fielder_id, bowling_team_id, season))
                                        full_wicket_id = f"{wicket_id_base}_{j}"
                                        wickets_rows.append((
                                            full_wicket_id, delivery_id, player_dismissed_id, dismissal_kind, fielder_id
                                        ))

                if deliveries_rows:
                    cursor.executemany("""
                        INSERT INTO deliveries (
                            delivery_id, match_id, inning_num, batting_team_id, bowling_team_id,
                            over_num, ball_num, batsman_id, bowler_id, non_striker_id,
                            runs_batsman, runs_extras, runs_total, extras_type, extras_runs
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, deliveries_rows)
                
                if powerplays:
                    cursor.executemany("""
                        INSERT INTO powerplay (powerplay_id, match_id, inning_num, start_over, end_over, pp_type)
                        VALUES (%s, %s, %s, %s, %s, %s)
                    """, powerplays)

                if reviews_rows:
                    cursor.executemany("""
                        INSERT INTO reviews (
                            review_id, match_id, delivery_id, review_by_team_id, umpire_id,
                            batsman_name_id, decision, review_type, umpires_call
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, reviews_rows)

                if replacements_rows:
                    cursor.executemany("""
                        INSERT INTO replacements (
                            replacement_id, match_id, team_id, player_in_id, player_out_id, reason
                        ) VALUES (%s, %s, %s, %s, %s, %s)
                    """, replacements_rows)

                if substitute_players_rows:
                    cursor.executemany("""
                        INSERT IGNORE INTO players (player_id, player_name, substitute)
                        VALUES (%s, %s, %s)
                    """, substitute_players_rows)

                if wickets_rows:
                    cursor.executemany("""
                        INSERT INTO wickets (
                            wicket_id, delivery_id, player_dismissed_id, dismissal_kind, fielder_id
                        ) VALUES (%s, %s, %s, %s, %s)
                    """, wickets_rows)

                if substitute_players_team:
                    cursor.executemany("""
                        INSERT IGNORE INTO players_team (player_id, team_id, season)
                        VALUES (%s, %s, %s)
                    """, substitute_players_team)
                

                print("Inserted DELIVERIES, REVIEWS, REPLACEMENTS & WICKET TABLE")

                # ========== OFFICIAL ROLE ==========
                current_section = "OFFICIAL ROLE"
                match_officials_rows = []
                for roles, names in officials_dict.items():
                    for off_name in names:
                        match_officials_rows.append((match_id, registry_people.get(off_name), roles))
                if match_officials_rows:
                    cursor.executemany("""
                        INSERT INTO match_officials (match_id, official_id, roles)
                        VALUES (%s, %s, %s)
                    """, match_officials_rows)
                    

                conn.commit()
                print(f"✅ Inserted {filename}")

            except mysql.connector.Error as e:
                conn.rollback()
                print(f"❌ MySQL Error in file {filename} → Section: {current_section} → {e}")
            except Exception as e:
                conn.rollback()
                print(f"❌ General Error in file {filename} → Section: {current_section} → {e}")

finally:
    conn.close()
    print("✅ Matches tables populated!")


Inserted TEAMS TABLE
Inserted PLAYERS & PLAYERS_TEAM TABLES
Inserted MATCH DETAILS TABLE
Inserted OFFICIALS TABLE
Inserted DELIVERIES, REVIEWS, REPLACEMENTS & WICKET TABLE
✅ Inserted 1473438.json
Inserted TEAMS TABLE
Inserted PLAYERS & PLAYERS_TEAM TABLES
Inserted MATCH DETAILS TABLE
Inserted OFFICIALS TABLE
Inserted DELIVERIES, REVIEWS, REPLACEMENTS & WICKET TABLE
✅ Inserted 1473439.json
Inserted TEAMS TABLE
Inserted PLAYERS & PLAYERS_TEAM TABLES
Inserted MATCH DETAILS TABLE
Inserted OFFICIALS TABLE
Inserted DELIVERIES, REVIEWS, REPLACEMENTS & WICKET TABLE
✅ Inserted 1473440.json
Inserted TEAMS TABLE
Inserted PLAYERS & PLAYERS_TEAM TABLES
Inserted MATCH DETAILS TABLE
Inserted OFFICIALS TABLE
Inserted DELIVERIES, REVIEWS, REPLACEMENTS & WICKET TABLE
✅ Inserted 1473441.json
Inserted TEAMS TABLE
Inserted PLAYERS & PLAYERS_TEAM TABLES
Inserted MATCH DETAILS TABLE
Inserted OFFICIALS TABLE
Inserted DELIVERIES, REVIEWS, REPLACEMENTS & WICKET TABLE
✅ Inserted 1473442.json
Inserted TEAMS TABLE